# Wikishop  
## Анализ комментариев на токсичность



In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot  as plt
%matplotlib inline


import re
import string


import torch
import transformers 

import optuna
import lightgbm as lgb


import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import f1_score

import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])


Error: Session cannot generate requests

In [185]:
!pip install torch

     |████████████████████████████████| 137.6 MB 36 kB/s 


In [187]:
pip install transformers

     |████████████████████████████████| 4.4 MB 301 kB/s 
     |████████████████████████████████| 3.6 MB 262 kB/s 
     |████████████████████████████████| 101 kB 255 kB/s 
Note: you may need to restart the kernel to use updated packages.


In [181]:
!pip install keras



     |████████████████████████████████| 1.6 MB 1.8 MB/s 


In [45]:
!python -m spacy download en_core_web_lg

     |██▎                             | 42.3 MB 412 kB/s eta 0:22:03^C

ERROR: Operation cancelled by user


In [20]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install spacy

     |████████████████████████████████| 6.5 MB 1.4 MB/s 
     |████████████████████████████████| 457 kB 3.4 MB/s 
     |████████████████████████████████| 768 kB 1.7 MB/s 
     |████████████████████████████████| 106 kB 1.2 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 181 kB 1.2 MB/s 
     |████████████████████████████████| 58 kB 1.4 MB/s 
     |████████████████████████████████| 5.9 MB 452 kB/s 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
path_global = '/datasets/toxic_comments.csv'
path_local = '/Users/ulia/Downloads/toxic_comments.csv'

if os.path.exists(path=path_global):
    data = pd.read_csv(path_global)
elif os.path.exists(path=path_local):
    data = pd.read_csv(path_local)
else: 
    print('ERROR IN PATH')

Error: Session cannot generate requests

In [5]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


### Text preprocessing

In [24]:
def  preprocess_text(text, stopwords):
    # cleaning punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # text to lower
    text = text.lower()

    # removing stopwords
    text = ' '.join([word for word in text.split() if word not in stopwords])

    # removing whitespaces
    text = re.sub(r'\s', ' ', text).strip()

    return text


    # 

In [ ]:
def  preprocess_text(text, stopwords):
    # cleaning punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # text to lower
    text = text.lower()

    # removing stopwords
    text = ' '.join([word for word in text.split() if word not in stopwords])

    # removing whitespaces
    text = re.sub(r'\s', ' ', text).strip()

    return text


In [73]:
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [27]:
data['clean'] = data['text'].apply(lambda x: preprocess_text(x, stopwords=stop_words))

In [74]:
data['lemmatized'] = data['clean'].apply(lambda x: lemmatize_text(x) )

In [76]:
data.head(20)

,text,toxic,clean,lemmatized
0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...,explanation edit make username hardcore metall...
1,D'aww! He matches this background colour I'm s...,0,daww matches background colour im seemingly st...,daww match background colour I m seemingly stu...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man im really trying edit war guy constant...,hey man I m really try edit war guy constantly...
3,"""\nMore\nI can't make any real suggestions on ...",0,cant make real suggestions improvement wondere...,can not make real suggestion improvement wonde...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page thats,sir hero chance remember page that s
5,"""\n\nCongratulations from me as well, use the ...",0,congratulations well use tools well · talk,congratulation well use tool well · talk
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker piss around work,cocksucker piss around work
7,Your vandalism to the Matt Shirvington article...,0,vandalism matt shirvington article reverted pl...,vandalism matt shirvington article revert plea...
8,Sorry if the word 'nonsense' was offensive to ...,0,sorry word nonsense offensive anyway im intend...,sorry word nonsense offensive anyway I m inten...
9,alignment on this subject and which are contra...,0,alignment subject contrary dulithgow,alignment subject contrary dulithgow


In [161]:
features = data['lemmatized']
target = data['toxic']

In [163]:
features_train_full, features_valid, target_train_full, target_valid = train_test_split(features, target, test_size=0.2, shuffle=True)
features_train, features_test, target_train, target_test = train_test_split(features_train_full, target_train_full, test_size=0.2, shuffle=True)

In [164]:
count_tf_idf = TfidfVectorizer()
features_train = count_tf_idf.fit_transform(features_train)

In [165]:
features_valid = count_tf_idf.transform(features_valid)
features_test = count_tf_idf.transform(features_test)

In [198]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
score = f1_score(predictions, target_valid)
score

0.7290957245554295

In [200]:
# Проверим на тестовой выборке:
score_test = f1_score(model.predict(features_test), target_test)


In [201]:
score_test

0.7269759850781068

In [169]:
test_sent = 'look at yourself, you are a fat, black, sodding idiot, thinking just about money'

In [170]:
test_sent_0 = 'I will surely recommend this product to my family and friends, thanks!'

In [171]:
test_sent = preprocess_text(test_sent, stop_words)
test_sent = lemmatize_text(test_sent)

In [172]:
test_sent_0 = preprocess_text(test_sent_0, stop_words)
test_sent_0 = lemmatize_text(test_sent_0)

In [173]:
test_sent_0

'surely recommend product family friend thank'

In [174]:
test = count_tf_idf.transform([test_sent])

In [176]:
test_0 = count_tf_idf.transform([test_sent_0])

In [178]:
model.predict(test)

array([1])

In [179]:
model.predict(test_0)

array([0])

### Наша модель не дотягивает до нужного уровня качества, попробуем по-другому

In [ ]:
#LightGBM
# попробуем применить optuna 
from optuna.integration import LightGBMPruningCallback
def objective(trial, features, target):
    param_grid = { 
        "n_estimators": trial.suggest_categorical("n_estimators", [100]),
        "num_leaves": trial.suggest_int("num_leaves", 2, 20, step=1),
        "max_depth": trial.suggest_int("max_depth", 5, 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3)
        }
    model = lgb.LGBMRegressor(**param_grid)
    model.fit(features_train, target_train, eval_set=[(features_valid, target_valid)],
    eval_metric='rmse')
    preds = model.predict(features_valid)
    rmse =  mean_squared_error(preds, target_valid, squared=False)
    return rmse

study =optuna.create_study(direction='maximize')
func = lambda trial: objective(trial, features_train, target_train)
study.optimize(func, n_trials=20)

Error: Session cannot generate requests

In [ ]:
print(f"\tBest value (rmse): {study.best_value:.2f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

Error: Session cannot generate requests

## Попробуем BERT

In [192]:
tokenizer = transformers.BertTokenizer(
    vocab_file='/Users/ulia/Downloads/uncased_L-8_H-512_A-8/vocab.txt')

tokenized = data['lemmatized'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [195]:
config = transformers.BertConfig.from_json_file(
    '/Users/ulia/Downloads/uncased_L-8_H-512_A-8/bert_config.json'
)

In [196]:
model_bert = transformers.BertModel.from_pretrained('/Users/ulia/Downloads/uncased_L-8_H-512_A-8/bert_model.ckpt.index', config=config)

OSError: Unable to load weights from pytorch checkpoint file for '/Users/ulia/Downloads/uncased_L-8_H-512_A-8/bert_model.ckpt.index' at '/Users/ulia/Downloads/uncased_L-8_H-512_A-8/bert_model.ckpt.index'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.